# 사용된 주요 코드 분석 및 테스트

### 📌 logging 모듈
- [Logging HOWTO](https://docs.python.org/3/howto/logging.html) 참조
- `coloredlogs` is a Python package that enhances the output of Python's logging module by adding colors to the log levels and timestamps.

In [18]:
import logging, coloredlogs

logger = logging.getLogger("my_logger")  # Create a logger object
coloredlogs.install(level="DEBUG", logger=logger)  # Install coloredlogs

# Log some messages
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning message")
logger.error("This is an error message")
logger.critical("This is a critical message")

2023-12-01 15:48:13 AERO15 my_logger[24180] DEBUG This is a debug message
2023-12-01 15:48:13 AERO15 my_logger[24180] INFO This is an info message
2023-12-01 15:48:13 AERO15 my_logger[24180] WARNING This is a warning message
2023-12-01 15:48:13 AERO15 my_logger[24180] ERROR This is an error message
2023-12-01 15:48:13 AERO15 my_logger[24180] CRITICAL This is a critical message


- Customizing `coloredlogs`

In [19]:
coloredlogs.install(
    level="DEBUG",
    logger=logger,
    fmt="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    datefmt="%H:%M:%S",
)

- Using coloredlogs with File Handlers

In [20]:
# File handler for logging to a file
file_handler = logging.FileHandler('my_log.log') # -> logging 모듈내에 있는 file handler 클래스로부터 instance object 생성
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')) # -> 생성된 핸들러 object의 포맷 설정
logger.addHandler(file_handler) # -> logger instance object에 파일 핸들러 object를 등록, 이후 로그메시지는 파일로 저장된다. (파일에는 color가 적용되지 않음)

- logger는 Namespace별 별도로 만들어 사용할 수 있다.

In [21]:
class attrdict(dict):  # dict의 모든 기능에 . 연산이 가능하도록 하는 dict 파생클래스 정의
    __getattr__ = dict.__getitem__  # 파이썬 언어의 first class 기능, 함수도 value처럼 사용 가능하다.
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


args = attrdict({"log": "debug"})  # attrdict(dict(log='debug'))도 가능


logger = logging.getLogger(__name__)  # __name__: 이 코드가 정의되는 Namespace 이름: 파이썬 데이터 모델 참조 (https://docs.python.org/3/reference/datamodel.html)
coloredlogs.install(
    level=args.log.upper(),  # print(f"args.log: {args.log.upper()}") 출력 참조
    logger=logger,
)

print(f"args.log: {args.log.upper()}")

args.log: DEBUG


#### 📌 Namespace 개념 : 일반적으로 Object 가 정의된 바로 상위(부모) 공간(boundary)을 말한다.
 - 파일이면 파일이 속한 폴더명이 Namespace
 - 파일내에 정의된 함수나 클래스는 해당 파일명이 Namespace
 - 함수안에 있는 로컬변수는 함수명이 Namespace
 - Class안에 있는 state 변수는 클래스명 Namespace
 - 객체 `AAA.BBB.CCC.DDD` 에서 `DDD`의 Namespace는 `CCC`가 된다.
 - 파이썬 코드에서 새로운(?) 객체(object)를 만나면, 타입이 무엇인지? 그 코드 라인의 Namespace가 어디인지 잘 이해해야 한다.
 - 사람은 폴더안에 무슨 파일이 있는지? 궁금해 하지만, Code는 자신이 실행 중 어떤 폴더(`Namespace`)에 있는가?를 아는 것이 메우 중요하다.

파이썬에서 네임스페이스(namespace)는 이름(식별자)이 객체(변수, 함수, 클래스 등)에 매핑되는 컬렉션입니다. 이는 프로그램 내에서 같은 이름을 여러 부분에서 다른 용도로 사용하려 할 때 충돌을 방지하기 위해 사용됩니다. 각 네임스페이스는 정의된 모든 이름을 추적하는 종류의 "주소록" 역할을 하며, 이름이 사용될 때 모호함이 없도록 보장합니다.

파이썬의 네임스페이스에 대한 몇 가지 중요한 측면은 다음과 같습니다:

##### 1. 네임스페이스의 종류:

- 로컬 네임스페이스(Local Namespace): 함수 내부의 지역 이름을 포함합니다. 이 네임스페이스는 함수가 호출될 때 생성되고 함수가 반환될 때까지만 지속됩니다.
- 글로벌 네임스페이스(Global Namespace): 현재 프로젝트에서 사용되는 다양한 임포트된 모듈의 이름을 포함합니다. 이 네임스페이스는 모듈이 읽혀질 때 생성되고 스크립트가 끝날 때까지 지속됩니다.
- 빌트인 네임스페이스(Built-in Namespace): 빌트인 함수와 예외 이름을 포함합니다.

##### 2. 네임스페이스의 수명: 
네임스페이스의 수명은 그것이 정의하는 객체의 범위에 따라 다릅니다. 예를 들어, 함수 내의 로컬 네임스페이스는 함수 호출이 끝나면 사라지지만, 글로벌 네임스페이스는 스크립트가 끝날 때까지 지속됩니다.

##### 3. 네임스페이스 접근: 
globals() 함수를 사용하여 글로벌 네임스페이스에 접근하고, locals() 함수를 사용하여 로컬 네임스페이스에 접근할 수 있습니다.

##### 4. 모듈은 네임스페이스로 작동: 
파이썬에서 각 모듈은 자체 글로벌 네임스페이스를 갖습니다. 이는 한 모듈에서 정의된 이름이 다른 모듈의 이름과 충돌하지 않음을 의미합니다. 일반적으로 이러한 이름은 점 표기법(예: module_name.variable_name)을 사용하여 접근합니다.

명시적인 네임스페이스 선언 부재: C++이나 C#과 같은 다른 언어들과 달리, 파이썬에는 네임스페이스를 선언하기 위한 명시적인 문법이 없습니다. 대신, 모듈, 클래스, 함수의 정의와 함께 암시적으로 생성됩니다.

##### 5. 범위 해결: 
파이썬은 이름 해결을 위해 LEGB 규칙(Local, Enclosing, Global, Built-in)을 따릅니다. 즉, 이름을 사용할 때 파이썬은 이 순서대로 네임스페이스를 검색하여 이름이 어디에 정의되어 있는지 찾습니다.
    
요약하자면, 파이썬에서의 네임스페이스는 프로그램 내에서 다양한 식별자를 조직하고 관리하는 핵심 개념으로, 이 식별자들의 사용에 있어서 모호함이 없도록 보장합니다.

### 📌 mediapipe
[구글 미디어파이프, Create the task 부분 참조](https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#create_the_task)   
- Live Stream 모드 코드 샘플   
```python
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

a = b = c = None 
# Create a hand landmarker instance with the live stream mode:
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    nonlocal a, b, c
    a = result
    b = output_image.numpy_view()
    c = timestamp_ms
    print('hand landmarker result: {}'.format(result))

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='/path/to/model.task'),  -> 미리 mediapipe 사이트에서 모델 파라미터를 다운받아 저장해 둔다. (e.g. models 폴더)
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)
with HandLandmarker.create_from_options(options) as landmarker:
    # The landmarker is initialized. Use it here.
    # ...
    
```

- Live stream mode는 async 방식을 사용한다. 콜백(callback) 함수(e.g. `print_result`)를 정의하여 mediapipe Detection Agent(e.g. `landmarker`)에 등록하면 Agent가 검출 완료시 자동으로 실행해 준다.
- `main_live.py` 코드 참조

In [22]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import logging, coloredlogs

try:
    from settings import *
    from utils import draw_fingercount_on_image
except:
    import sys
    sys.path.insert(0, "..") # 상위폴더를 import search path에 추가, cwd(current working directory) is 'notebook', not 'OSS-PROJECT-13'
    from settings import *
    from utils import draw_fingercount_on_image

logger = logging.getLogger(__name__)
coloredlogs.install(level="DEBUG", logger=logger)  # logger 설정

cap = cv2.VideoCapture(0)

# https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#video
base_options = python.BaseOptions(model_asset_path=root / "models/hand_landmarker.task")
options = vision.HandLandmarkerOptions(base_options=base_options, num_hands=2)
with vision.HandLandmarker.create_from_options(options) as detector: # -> 'detector' context 범위, similar to file operation
    while True:
        success, image_origin = cap.read()

        if success:
            image = cv2.cvtColor(image_origin, cv2.COLOR_BGR2RGB)
            rgb_frame = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
            detection_result = detector.detect(rgb_frame)  # notebook/HandLandmarkerResult.ipynb 참조

            if detection_result:  # detection_result 로그데이터 생성
                logger.debug(f"detection_result:\n {detection_result}")

            annotated_image = draw_fingercount_on_image(  # 원본 카메라 영상에 detection 결과를 합성
                rgb_image=image_origin,
                detection_result=detection_result,
            )

            cv2.imshow(f"Finger Counter", annotated_image)  # 동영상 재생

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            logger.critical(f"cap.read() error")
            break

    cap.release()
    cv2.destroyAllWindows()


2023-12-01 15:48:16 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:16 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:16 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:16 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:16 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:17 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_landmarks=[], hand_world_landmarks=[])
2023-12-01 15:48:17 AERO15 __main__[24180] DEBUG detection_result:
 HandLandmarkerResult(handedness=[], hand_lan

### 📌 utils.py

### Namespace 속성(or member) 검출 함수: dir()
 - python built-in 함수
 - namespace(모듈 또는 object 내부)에 있는 멤버들의 이름(str)을 리턴해 준다.
 - module 또는 object 분석에 편리함

In [23]:
dir(list) # list 클래스가 가지고 있는 모든 속성명칭을 리턴한다

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

멤버들의 이름으로 부터 정의된 value를 읽을 때 `getattr` built-in 함수를 이용한다.

In [24]:
for k in dir(list):
    v = getattr(list, k)
    print(f"list.{k}: {v}")

list.__add__: <slot wrapper '__add__' of 'list' objects>
list.__class__: <class 'type'>
list.__class_getitem__: <built-in method __class_getitem__ of type object at 0x00007FF971784D00>
list.__contains__: <slot wrapper '__contains__' of 'list' objects>
list.__delattr__: <slot wrapper '__delattr__' of 'object' objects>
list.__delitem__: <slot wrapper '__delitem__' of 'list' objects>
list.__dir__: <method '__dir__' of 'object' objects>
list.__doc__: Built-in mutable sequence.

If no argument is given, the constructor creates a new empty list.
The argument must be an iterable if specified.
list.__eq__: <slot wrapper '__eq__' of 'list' objects>
list.__format__: <method '__format__' of 'object' objects>
list.__ge__: <slot wrapper '__ge__' of 'list' objects>
list.__getattribute__: <slot wrapper '__getattribute__' of 'list' objects>
list.__getitem__: <method '__getitem__' of 'list' objects>
list.__getstate__: <method '__getstate__' of 'object' objects>
list.__gt__: <slot wrapper '__gt__' of 'l

In [25]:
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

In [26]:
for k in dir(HandLandmarker): # Package folder의 내부 구성요소 검색
    v = getattr(HandLandmarker, k)
    print(f"HandLandmarker.{k}: {v}")

HandLandmarker.__class__: <class 'type'>
HandLandmarker.__delattr__: <slot wrapper '__delattr__' of 'object' objects>
HandLandmarker.__dict__: {'__module__': 'mediapipe.tasks.python.vision.hand_landmarker', '__doc__': 'Class that performs hand landmarks detection on images.', 'create_from_model_path': <classmethod(<function HandLandmarker.create_from_model_path at 0x000001A618C8E980>)>, 'create_from_options': <classmethod(<function HandLandmarker.create_from_options at 0x000001A618C8EA20>)>, 'detect': <function HandLandmarker.detect at 0x000001A618C8EAC0>, 'detect_for_video': <function HandLandmarker.detect_for_video at 0x000001A618C8EB60>, 'detect_async': <function HandLandmarker.detect_async at 0x000001A618C8EC00>}
HandLandmarker.__dir__: <method '__dir__' of 'object' objects>
HandLandmarker.__doc__: Class that performs hand landmarks detection on images.
HandLandmarker.__enter__: <function BaseVisionTaskApi.__enter__ at 0x000001A618B99EE0>
HandLandmarker.__eq__: <slot wrapper '__eq_

In [2]:
try:
    import algorithm
except:
    import sys
    sys.path.insert(0, "..")
    import algorithm

for k in dir(algorithm): # algorithm.py 파일내 검색
    try: # 코드에서 Exception 에러가 발생해도 for loop 끝까지 수행하도록
        v = getattr(algorithm, k)
        if callable(v):   # 호출() 동작이 가능하면,  또 다른 예를 들면, isinstance, issubclass 등의 기능도 있다
            print(f"algorithm.{k}: {v}")
    except:
        continue

2023-12-02 03:56:00 AERO15 settings[18528] INFO start from the settings

2023-12-02 03:56:00 AERO15 settings[18528] WARNING 폴더경로에 한글이 있으면 model parameter 로드시 error가 발생할 수 있습니다.
2023-12-02 03:56:00 AERO15 settings[18528] INFO project root folder: c:\Users\kclee\OneDrive\Develope\2023-2\OpenSW\OSS-project-13_Master\notebook\..



algorithm.Algorithm: <class 'algorithm.Algorithm'>
algorithm.Basic: <class 'algorithm.Basic'>
algorithm.Category: <class 'mediapipe.tasks.python.components.containers.category.Category'>
algorithm.Curvature: <class 'algorithm.Curvature'>
algorithm.HandLandmarkerResult: <class 'mediapipe.tasks.python.vision.hand_landmarker.HandLandmarkerResult'>
algorithm.Landmark: <class 'mediapipe.tasks.python.components.containers.landmark.Landmark'>
algorithm.NormalizedLandmark: <class 'mediapipe.tasks.python.components.containers.landmark.NormalizedLandmark'>
algorithm.ThumbCheck: <class 'algorithm.ThumbCheck'>
algorithm.distance: <function distance at 0x0000019FF0A42660>
algorithm.radius: <function radius at 0x0000019FF08D5F80>
algorithm.to_list_landmarks: <function to_list_landmarks at 0x0000019FF0A40860>
algorithm.to_ndarray: <function to_ndarray at 0x0000019FF08D6700>


In [3]:
try:
    import algorithm
except:
    import sys
    sys.path.insert(0, "..")
    import algorithm

for k in dir(algorithm): # algorithm.py 파일내 알고리즘 검색
    try: # 코드에서 Exception 에러가 발생해도 for loop 끝까지 수행하도록
        v = getattr(algorithm, k)
        if issubclass(v, algorithm.Algorithm) and v is not algorithm.Algorithm:
            print(f"algorithm.{k}: {v}")
    except TypeError:
        continue

algorithm.Basic: <class 'algorithm.Basic'>
algorithm.Curvature: <class 'algorithm.Curvature'>
algorithm.ThumbCheck: <class 'algorithm.ThumbCheck'>


### 📌 argparse: command line argument 읽는 법

파이썬 Howto [Argparse Tutorial](https://docs.python.org/3/howto/argparse.html) 문서 참조

command line 에서 `python main.py --help` 를 입력하면,
```bash
❯ python main.py --help
usage: main.py [-h] [--algo {basic,curvature,thumbcheck}] [--log {debug,info,warning,error,critical}] [--log_a {debug,info,warning,error,critical}] [--save]

options:
  -h, --help            show this help message and exit
  --algo {basic,curvature,thumbcheck}  -> 목록 가운데 하나를 선택입력, default값 basic
                        select finger counter algorithm
  --log {debug,info,warning,error,critical} -> 목록 가운데 하나를 선택하여 입력, default값 info
                        select log level 
  --log_a {debug,info,warning,error,critical} -> 목록 가운데 하나를 선택하여 입력, default값 info
                        select algorithm log level
  --save                  -> 지정하면 video 영상을 자동으로 저장
                        save video data as .avi file
```

프로그램 동작
```python
args = get_args()  # 프로그램 실행시 command line 에서 입력된 argument 값을 읽어 args에 설정하는 코드
```

```python
def get_args():
    algos = extract_subclass(algorithm, algorithm.Algorithm)   -> algorithm.py 파일에서 Algorithm 파생클래스를 찾아 목록(list[dict])으로 구성
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--algo",
        help="select finger counter algorithm",
        default="thumbcheck",
        type=str,
        required=False,
        choices=list(algos.keys()), -> Algorithm 파생클래스 이름(key값)을 목록으로 변환하여 선택리스트에 전달한다.
    )
    parser.add_argument(
        "--log",
        help="select log level",
        default="info",
        type=str,
        required=False,
        choices=["debug", "info", "warning", "error", "critical"],
    )
    parser.add_argument(
        "--log_a",
        help="select algorithm log level",
        default="info",
        type=str,
        required=False,
        choices=["debug", "info", "warning", "error", "critical"],
    )
    parser.add_argument(
        "--save",
        help="save video data as .avi file",
        default=False,
        action="store_true",
        required=False,
    )

    args = parser.parse_args()  -> 위에서 정의된 Argument를 기반으로 command line에서 입력된 argument 값을 가져온다

    return args

```

🚩 args 의 필드값
```python
 - args.algo: str
 - args.log: str
 - args.log_a: str
 - args.save: bool
 
```

🚩 예) 로그레벨 설정 코드
```python
logger = logging.getLogger(__name__)  -> 파이썬 모든 코드 영역에는 자신이 속한 Namepace가 정의되어 있다. 그 Namespace 이름은 __name__ 을 사용하여 가져온다. 
                                         파이썬 데이터 모델 (https://docs.python.org/3/reference/datamodel.html) 참조
coloredlogs.install(level=args.log.upper(), logger=logger)  -> args.log 값을 읽어 대문자로 변환하여 설정함을 알 수 있다
```


#### 📌 Curvature Calculation

In [29]:
import numpy as np

In [30]:
def radius_center(A, B, C):

    # A = np.array([2.0, 1.5, 0.0])
    # B = np.array([6.0, 4.5, 0.0])
    # C = np.array([11.75, 6.25, 0.0])
    a = np.linalg.norm(C - B)
    b = np.linalg.norm(C - A)
    c = np.linalg.norm(B - A)
    s = (a + b + c) / 2
    radius = a*b*c / 4 / np.sqrt(s * (s - a) * (s - b) * (s - c))
    b1 = a*a * (b*b + c*c - a*a)
    b2 = b*b * (a*a + c*c - b*b)
    b3 = c*c * (a*a + b*b - c*c)
    center = np.column_stack((A, B, C)).dot(np.hstack((b1, b2, b3)))
    center /= b1 + b2 + b3

    return (radius, center) if radius > 0 else (np.inf, np.array([np.inf, np.inf, np.inf]))


# print(R)
# print(P)

# 15.899002930062531
# array([ 13.42073171,  -9.56097561,   0.        ])

### hand_landmarks(21개 핸드 특징점 좌표) 데이터 분석

In [1]:
import numpy as np
from mediapipe.tasks.python.vision.hand_landmarker import HandLandmarkerResult
from mediapipe.tasks.python.components.containers.category import Category
from mediapipe.tasks.python.components.containers.landmark import NormalizedLandmark, Landmark

results = HandLandmarkerResult(
    handedness=[[Category(index=1, score=0.9926292896270752, display_name="Left", category_name="Left")]],
    hand_landmarks=[
        [
            NormalizedLandmark(x=0.6419628858566284, y=0.616515040397644, z=-6.035641604285047e-07, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5716654062271118, y=0.6108736395835876, z=-0.002957998076453805, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5272430777549744, y=0.5887590646743774, z=-0.01388684194535017, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4818960428237915, y=0.5766850113868713, z=-0.029851868748664856, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.44992589950561523, y=0.5541561841964722, z=-0.04740559309720993, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6141445636749268, y=0.43760523200035095, z=0.007367989979684353, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5445273518562317, y=0.428905725479126, z=-0.007748082280158997, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5036388635635376, y=0.45918241143226624, z=-0.025927452370524406, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.47835105657577515, y=0.49549776315689087, z=-0.03698492795228958, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.6072747707366943, y=0.42586055397987366, z=-0.00027601292822510004, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5201941132545471, y=0.42209744453430176, z=-0.009670299477875233, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4791445732116699, y=0.4669061303138733, z=-0.02469821274280548, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.46002593636512756, y=0.5104443430900574, z=-0.03566863387823105, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5915679335594177, y=0.4281168580055237, z=-0.009448129683732986, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5056634545326233, y=0.4338810443878174, z=-0.017011014744639397, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4680931866168976, y=0.4771495461463928, z=-0.02728145755827427, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4502561092376709, y=0.516810953617096, z=-0.034487128257751465, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.5680385231971741, y=0.4449925422668457, z=-0.019567394629120827, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.504776656627655, y=0.45213910937309265, z=-0.022505883127450943, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.47540536522865295, y=0.47587350010871887, z=-0.023087508976459503, visibility=0.0, presence=0.0),
            NormalizedLandmark(x=0.4563266634941101, y=0.5018525123596191, z=-0.022691285237669945, visibility=0.0, presence=0.0),
        ]
    ],
    hand_world_landmarks=[
        [
            Landmark(x=0.009037008509039879, y=0.06189635396003723, z=-0.03779924288392067, visibility=0.0, presence=0.0),
            Landmark(x=-0.0033948568161576986, y=0.05620351806282997, z=-0.012544753029942513, visibility=0.0, presence=0.0),
            Landmark(x=-0.02560390532016754, y=0.04868587106466293, z=0.014155255630612373, visibility=0.0, presence=0.0),
            Landmark(x=-0.04892528057098389, y=0.04613838344812393, z=0.05510091409087181, visibility=0.0, presence=0.0),
            Landmark(x=-0.06683654338121414, y=0.034338757395744324, z=0.08791903406381607, visibility=0.0, presence=0.0),
            Landmark(x=-0.0019140357617288828, y=0.000755488988943398, z=0.01971326768398285, visibility=0.0, presence=0.0),
            Landmark(x=-0.021188266575336456, y=-0.0017718548187986016, z=0.027694974094629288, visibility=0.0, presence=0.0),
            Landmark(x=-0.037010885775089264, y=0.001074992585927248, z=0.03401287645101547, visibility=0.0, presence=0.0),
            Landmark(x=-0.05434488505125046, y=0.01770664006471634, z=0.030251258984208107, visibility=0.0, presence=0.0),
            Landmark(x=0.00110423075966537, y=-0.003265355248004198, z=0.007934767752885818, visibility=0.0, presence=0.0),
            Landmark(x=-0.028509020805358887, y=-0.013048581779003143, z=0.018052339553833008, visibility=0.0, presence=0.0),
            Landmark(x=-0.0526660792529583, y=0.0021621340420097113, z=0.023894505575299263, visibility=0.0, presence=0.0),
            Landmark(x=-0.06124870851635933, y=0.01557418704032898, z=0.03212356194853783, visibility=0.0, presence=0.0),
            Landmark(x=-0.0001015738962450996, y=-0.0032526326831430197, z=-0.01480625756084919, visibility=0.0, presence=0.0),
            Landmark(x=-0.029653087258338928, y=-0.0027253220323473215, z=0.0009754223865456879, visibility=0.0, presence=0.0),
            Landmark(x=-0.05461142584681511, y=0.00921244453638792, z=0.011858618818223476, visibility=0.0, presence=0.0),
            Landmark(x=-0.0628863126039505, y=0.022142725065350533, z=0.02059471420943737, visibility=0.0, presence=0.0),
            Landmark(x=-0.01317686215043068, y=0.010379736311733723, z=-0.027534576132893562, visibility=0.0, presence=0.0),
            Landmark(x=-0.02998003363609314, y=0.0076247043907642365, z=-0.01695789210498333, visibility=0.0, presence=0.0),
            Landmark(x=-0.049735985696315765, y=0.005921175237745047, z=-0.008792607113718987, visibility=0.0, presence=0.0),
            Landmark(x=-0.05839437246322632, y=0.01687566563487053, z=4.301162698538974e-05, visibility=0.0, presence=0.0),
        ]
    ],
)

In [2]:
landmarks = np.array(results.hand_landmarks[0])
landmarks

array([NormalizedLandmark(x=0.6419628858566284, y=0.616515040397644, z=-6.035641604285047e-07, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5716654062271118, y=0.6108736395835876, z=-0.002957998076453805, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5272430777549744, y=0.5887590646743774, z=-0.01388684194535017, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.4818960428237915, y=0.5766850113868713, z=-0.029851868748664856, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.44992589950561523, y=0.5541561841964722, z=-0.04740559309720993, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.6141445636749268, y=0.43760523200035095, z=0.007367989979684353, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5445273518562317, y=0.428905725479126, z=-0.007748082280158997, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5036388635635376, y=0.45918241143226624, z=-0.025927452370524406, visibility=0.0, presence

In [8]:
import pandas as pd

data = []
for mark in landmarks:
    data.append([mark.x, mark.y, mark.z])
data

df = pd.DataFrame(data, columns=['x', 'y', 'z'])
df.csv


,x,y,z
0,0.641963,0.616515,-6.035642e-07
1,0.571665,0.610874,-2.957998e-03
2,0.527243,0.588759,-1.388684e-02
3,0.481896,0.576685,-2.985187e-02
4,0.449926,0.554156,-4.740559e-02
5,0.614145,0.437605,7.367990e-03
6,0.544527,0.428906,-7.748082e-03
7,0.503639,0.459182,-2.592745e-02
8,0.478351,0.495498,-3.698493e-02
9,0.607275,0.425861,-2.760129e-04


In [34]:
def to_ndarray(landmark):
    return np.array([landmark.x, landmark.y, landmark.z])

In [35]:
to_ndarray(landmarks[0])

array([ 6.41962886e-01,  6.16515040e-01, -6.03564160e-07])

In [36]:
r, c = radius_center(
    to_ndarray(landmarks[0]),
    to_ndarray(landmarks[1]),
    to_ndarray(landmarks[4]),
)
r

0.24047688819905672

In [37]:
landmarks[[0, 1, 2]]

array([NormalizedLandmark(x=0.6419628858566284, y=0.616515040397644, z=-6.035641604285047e-07, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5716654062271118, y=0.6108736395835876, z=-0.002957998076453805, visibility=0.0, presence=0.0),
       NormalizedLandmark(x=0.5272430777549744, y=0.5887590646743774, z=-0.01388684194535017, visibility=0.0, presence=0.0)],
      dtype=object)